# Data from UNESCO

Download csvs from ILO's SDMX API (ILOSTAT SDMX Gateway). 

### Load required libraries

In [1]:
import json
import requests 
import os
from bs4 import BeautifulSoup
import re
import pandas as pd

### Create folder for data

In [2]:
#os.mkdir('..\\data')

### Use email (old indicators) and dictionary to create new indicators

In [4]:
df = pd.read_excel('..\\data\\UNESCO_UIS_indicators\\uis-data-dictionary-education-statistics.xlsx',
                   sheet_name = 'Students and Teachers')
idctr = pd.read_csv('..\\data\\UNESCO_UIS_indicators\\email.csv')
idctr.Old_Code = [i.replace('_','.') for i in idctr.Old_Code]
New_list = []
for i in idctr.Old_Code:
    a = 'UNESCO,EDU_NON_FINANCE,3.0/'
    for s in df[df["Indicator ID"].str.lower() == i.lower()].iloc[0,6:-2]:
        a = a  + str(s) +'.'
    New_list.append(a)
idctr['New_Code'] = New_list
idctr


,GSMI,Indicator,Disaggregation,Old_Code,New_Code
0,20,20. Youth literacy rate of persons (15-24 year...,Both_sexes,LR.Ag15t24,"UNESCO,EDU_NON_FINANCE,3.0/LR.PT._Z._Z._T.Y15T..."
1,20,20. Youth literacy rate of persons (15-24 year...,Female,LR.Ag15t24.F,"UNESCO,EDU_NON_FINANCE,3.0/LR.PT._Z._Z.F.Y15T2..."
2,20,20. Youth literacy rate of persons (15-24 year...,Male,LR.Ag15t24.M,"UNESCO,EDU_NON_FINANCE,3.0/LR.PT._Z._Z.M.Y15T2..."
3,20,20. Youth literacy rate of persons (15-24 year...,GPI,LR.Ag15t24.gpi,"UNESCO,EDU_NON_FINANCE,3.0/LR.GPI._Z._Z._T.Y15..."
4,21,21. Adjusted net enrolment rate in primary edu...,Both_sexes,NERA.1.cp,"UNESCO,EDU_NON_FINANCE,3.0/NERA.PT.L1._T._T.SC..."
5,21,21. Adjusted net enrolment rate in primary edu...,Female,NERA.1.F.cp,"UNESCO,EDU_NON_FINANCE,3.0/NERA.PT.L1._T.F.SCH..."
6,21,21. Adjusted net enrolment rate in primary edu...,Male,NERA.1.M.cp,"UNESCO,EDU_NON_FINANCE,3.0/NERA.PT.L1._T.M.SCH..."
7,21,21. Adjusted net enrolment rate in primary edu...,GPI,NERA.1.gpi,"UNESCO,EDU_NON_FINANCE,3.0/NERA.GPI.L1._T._T.S..."
8,22,22.Gross enrolment ratio in secondary educatio...,Both_sexes,GER.2t3,"UNESCO,EDU_NON_FINANCE,3.0/GER.PT.L2_3._T._T.S..."
9,22,22.Gross enrolment ratio in secondary educatio...,Female,GER.2t3.F,"UNESCO,EDU_NON_FINANCE,3.0/GER.PT.L2_3._T.F.SC..."


### Interactive shell

In [5]:
# https://volderette.de/jupyter-notebook-tip-multiple-outputs/
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [6]:
############### put your api key here ####################
key = 'eacdf60587ba47a092eaf020b4bd42f5'



headers ={'Ocp-Apim-Subscription-key': key}

### find countryCode through parsing CL_codelist

In [7]:
url = 'https://api.uis.unesco.org/sdmx/codelist/UNESCO/CL_AREA/latest?format=sdmx-compact-2.1' #contain 3-
# creating HTTPS response object from given url 
resp = requests.get(url,headers = headers) 

soup = BeautifulSoup(resp.content)

longlist = soup.find_all('str:code')

countryCodes = [l for l in [l['id'] for l in longlist]]

len(countryCodes) #234

374

In [8]:
len(countryCodes) 

374

### Get data from API endpoint and save as xml file on disk

In [9]:
def buildUrl(indicatorCode):
    baseURL = 'https://api.uis.unesco.org/sdmx/data/'
    url = baseURL +  indicatorCode + '?startPeriod=2000&endPeriod=2018&format=csv&locale=en'  # 
    return url

In [10]:
def getcsv (url, file):
    # creating HTTP response object from given url 
    resp = requests.get(url, headers = headers) 
    file_loc = '..\\data\\UNESCO_UIS_indicators\\' + file
    # saving the xml message into an xml file 
    with open(file_loc, 'wb') as f: 
        f.write(resp.content) 
    display('finished downloading ' + file)
        

In [11]:

for i in range(55):
    url = buildUrl(idctr.New_Code[i])

    file = idctr.Indicator[i] + ' ' + idctr.Disaggregation[i] + '.csv'

    getcsv(url,file)

'finished downloading 20. Youth literacy rate of persons (15-24 years), by sex Both_sexes.csv'

'finished downloading 20. Youth literacy rate of persons (15-24 years), by sex Female.csv'

'finished downloading 20. Youth literacy rate of persons (15-24 years), by sex Male.csv'

'finished downloading 20. Youth literacy rate of persons (15-24 years), by sex GPI.csv'

'finished downloading 21. Adjusted net enrolment rate in primary education by sex Both_sexes.csv'

...

'finished downloading 31d. Cumulative Educational attainment (minimum post-secondary) of the population aged 25 and older, by sex GPI.csv'

'finished downloading 31e. Cumulative Educational attainment (minimum tertiary) of the population aged 25 and older, by sex Both_sexes.csv'

'finished downloading 31e. Cumulative Educational attainment (minimum tertiary) of the population aged 25 and older, by sex Female.csv'

'finished downloading 31e. Cumulative Educational attainment (minimum tertiary) of the population aged 25 and older, by sex Male.csv'

'finished downloading 31e. Cumulative Educational attainment (minimum tertiary) of the population aged 25 and older, by sex GPI.csv'